In [1]:
import collections
import os
import tensorflow as tf
import pandas as pd

tf.__version__


'2.13.0'

In [2]:
main_dir = r"F:\linux_ubuntu_codes\codes\nlp_lecture"

In [3]:
train_dir = os.path.join(main_dir + r"\train.csv")
test_dir = os.path.join(main_dir + r"\test.csv")

'https://raw.githubusercontent.com/sedeba19/NLP--Random-Models/main/train.csv'

In [4]:
train_dir

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
# Make a train_df
train_df = pd.read_csv(train_dir)
test_df = pd.read_csv(test_dir)
train_df.head()

,id,keyword,location,text,target
3228,4632,emergency%20services,"Sydney, New South Wales",Goulburn man Henry Van Bilsen missing: Emergen...,1
3706,5271,fear,NaN,The things we fear most in organizations--fluc...,0
6957,9982,tsunami,Land Of The Kings,@tsunami_esh ?? hey Esh,0
2887,4149,drown,NaN,@POTUS you until you drown by water entering t...,0
7464,10680,wounds,"cody, austin follows ?*?",Crawling in my skin\nThese wounds they will no...,1


In [6]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac = 1, random_state = 1)
train_df_shuffled.head()

(7613, 5)

In [7]:
train_df_shuffled.shape

target
0    4342
1    3271
Name: count, dtype: int64

In [8]:
# How many examples of each class?
train_df['target'].value_counts()

(7613, 3263)

In [9]:
# How many total samples?
len(train_df), len(test_df)

Target:0 (not a real disaster)
Text: 
@briannafrost Twister with Bill Paxton and Helen Hunt!
-----

Target:0 (not a real disaster)
Text: 
Pretty teen Hayden Ryan poses and strips out of her purple top http://t.co/qew4c5M1xd View and download video
-----

Target:0 (not a real disaster)
Text: 
#hot  Funtenna: hijacking computers to send data as sound waves [Black Hat 2015] http://t.co/s4PNIhJQX7 #prebreak #best
-----

Target:0 (not a real disaster)
Text: 
I get this feeling that society will collapse or implode. So don't be a hero and play your part.
-----

Target:0 (not a real disaster)
Text: 
We offer #preventative services such as cabling and bracing to stop structural failure of your trees. Call 905-877-8591 to book yours. #CTS
-----


In [ ]:
# Visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index: random_index +5].itertuples():
    _, text, target = row
    if target > 0:
        print(f"Target:{target}", "(real disaster)")
    else:
        print(f"Target:{target}", "(not a real disaster)")
    
    print(f"Text: \n{text}")
    print("-----\n")

### Split data into training and validation sets

In [11]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df['text'],
                                                                            train_df['target'],
                                                                            test_size = 0.1,
                                                                            random_state = 42)

(7613,)

In [12]:
train_df['text'].shape

0       Our Deeds are the Reason of this #earthquake M...
1                  Forest fire near La Ronge Sask. Canada
2       All residents asked to 'shelter in place' are ...
3       13,000 people receive #wildfires evacuation or...
4       Just got sent this photo from Ruby #Alaska as ...
                              ...                        
7608    Two giant cranes holding a bridge collapse int...
7609    @aria_ahrary @TheTawniest The out of control w...
7610    M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611    Police investigating after an e-bike collided ...
7612    The Latest: More Homes Razed by Northern Calif...
Name: text, Length: 7613, dtype: object

In [13]:
train_df["text"]

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       ...,
       'M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ',
       'Police investigating after an e-bike collided with a car in Little Portugal. E-bike rider suffered serious non-life threatening injuries.',
       'The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d'],
      dtype=object)

In [14]:
train_df['text'].to_numpy()

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       ...,
       'M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ',
       'Police investigating after an e-bike collided with a car in Little Portugal. E-bike rider suffered serious non-life threatening injuries.',
       'The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d'],
      dtype=object)

In [15]:
train_df['text'].to_numpy()

(6851, 762, 6851, 762)

In [ ]:
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

### Converting text into numbers: Text Vectorization also known as Tokenization

In [17]:
# Get the total words
total_words = 0
for i in train_sentences:
    total_words += len(i.split())

total_words

(6851,)

In [18]:
train_sentences.shape

15

In [19]:
# Get the average words per sentence or line
avg_words_per_sentence = round(total_words/len(train_sentences))
avg_words_per_sentence

In [20]:
# Normally, total number of words is equal or less than the max vocabulary length
max_vocab_length = 10000

# max length of a statement is equal or more than the average word per sentence or line
max_length = 15

In [21]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode = "int",
                                    output_sequence_length = max_length)

# Fit the text vectorizer to the training set
text_vectorizer.adapt(train_sentences)

There's a flood in my street! Sentence length is 29.


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[282,   3, 206,   4,  13, 674,   0,   0,   0,   0,   0,   0,   0,
          0,   0]], dtype=int64)>

In [22]:
# Create a sample sentence and tokenize it
samp_sentence = "There's a flood in my street!"
print(samp_sentence, f"Sentence length is {len(samp_sentence)}.")
text_vectorizer([samp_sentence])
#len(tf.squeeze(text_vectorizer([samp_sentence])))

Original text: 

 Spot Flood Combo 53inch 300W Curved Cree LED Work Light Bar 4X4 Offroad Fog Lamp - Full reÛ_ http://t.co/jCDd6SD6Qn http://t.co/9gUCkjghms 

Vectorized text:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 758,  206, 1548, 2199, 2203, 2139, 1083,  939,  153,  625,  694,
        1919, 1110, 1847, 1062]], dtype=int64)>

In [23]:
# Choose random sentence from the training dataset and tokenize it
rand_train_sentence = random.choice(train_sentences)
print(f"Original text: \n\n {rand_train_sentence} \n\nVectorized text:")
text_vectorizer([rand_train_sentence])

['',
 '[UNK]',
 'the',
 'a',
 'in',
 'to',
 'of',
 'and',
 'i',
 'is',
 'for',
 'on',
 'you',
 'my',
 'with',
 'it',
 'that',
 'at',
 'by',
 'this',
 'from',
 'are',
 'be',
 'was',
 'have',
 'like',
 'as',
 'me',
 'but',
 'up',
 'just',
 'so',
 'im',
 'not',
 'amp',
 'your',
 'out',
 'all',
 'after',
 'its',
 'has',
 'no',
 'will',
 'an',
 'fire',
 'when',
 'if',
 'we',
 'get',
 'now',
 'more',
 'via',
 'new',
 'about',
 'what',
 'people',
 'or',
 'news',
 'he',
 'they',
 'over',
 'one',
 'been',
 'how',
 'dont',
 'who',
 'video',
 'into',
 'were',
 'do',
 'us',
 'can',
 'emergency',
 '2',
 'disaster',
 'there',
 'her',
 'some',
 'than',
 'still',
 'would',
 'his',
 'police',
 'crash',
 'burning',
 'suicide',
 'body',
 'california',
 'back',
 'time',
 'buildings',
 'had',
 'why',
 'off',
 'them',
 'got',
 'man',
 'see',
 'storm',
 'know',
 'going',
 'cant',
 'world',
 'first',
 'day',
 'rt',
 'nuclear',
 'love',
 'youtube',
 'our',
 'attack',
 'go',
 'fires',
 'two',
 'their',
 'bomb',

In [24]:
# Get unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
words_in_vocab

10000

In [25]:
len(words_in_vocab)

['', '[UNK]', 'the', 'a', 'in']

In [26]:
# Top 5 words in words_in_vocab
words_in_vocab[:5]

['pakthey', 'pakistan\x89Ûªs', 'pakistans', 'pajamas', 'paints']

In [ ]:
# Least 5 words in words_in_vocab
words_in_vocab[-5:]

### Embedding layer

To make our embedding, we are going to use TensorFlow’s embedding layer: https://www.tensorflow.org/api/docs/python/tf/keras/layers/

The parameters are below mostly used.
* input_dim = size of the vocabulary
* output_dim = size of the output embedding vector.  For example, a value of 100 would mean each token gets represented by a vector 100 long
* input_length = length of the sequences being passed to the embedding layer

In [28]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim = max_vocab_length,
                             output_dim = 64,
                             embeddings_initializer = "uniform",
                             input_length = max_length)

Original text: The Latest: More homes razed by Northern California wildfire - http://t.co/ecXMoiNZgU http://t.co/ntwWDweDNb

Vectorized text: [[  2 199  50 200 480  18 184  87 157   1   1   0   0   0   0]]

Vectorized text shape: (1, 15)
Embbedded text: [[[-0.02597304  0.00594245 -0.0247831  -0.0028858  -0.0467329
    0.00426599  0.04692217 -0.03798555  0.02209764 -0.00445684
   -0.02018706  0.03898202 -0.00934716  0.01586461  0.04437922
   -0.02081614  0.00849216  0.01618148 -0.01961809  0.02844448
   -0.029401    0.00175206  0.03839587  0.0047024   0.02103131
   -0.00418274  0.04071205  0.00492196  0.02227474  0.01864977
   -0.03042555 -0.04731048 -0.02336736 -0.01034587  0.04365828
    0.01576695 -0.01080794 -0.02639517  0.02581752 -0.01493729
    0.00569297 -0.01363313  0.01970481 -0.01598551 -0.03030607
    0.00402067  0.00356252  0.01800628 -0.03250928  0.03349555
    0.00037335  0.04589185 -0.04598429  0.00094364 -0.04623948
   -0.01347191  0.00787818  0.04431866 -0.0123047  -0.

In [ ]:
# Get a random sentence from the training set and return Original text, Vectorized text, Embedded text
rand_sentence = random.choice(train_sentences)
print(f"Original text: {rand_sentence}\n")
print(f"Vectorized text: {text_vectorizer([rand_sentence])}\n")
print(f"Vectorized text shape: {text_vectorizer([rand_sentence]).shape}")
print(f"Embbedded text: {embedding(text_vectorizer([rand_sentence]))}")
print(f"Embedded text shape: {embedding(text_vectorizer([rand_sentence])).shape}")

## Model_7: TensorFlow Hub Pretrained Feature Extractor (10% of Data)

In [30]:
# Create subsets of 10% of the training and label data
train_10_percent = train_df[['text', "target"]].sample(frac= 0.1, random_state=42)
train_sentences_train_10_percent = train_10_percent["text"].to_list()
train_labels_train_10_percent = train_10_percent["target"].to_list()

(761, 761)

In [31]:
len(train_sentences_train_10_percent), len(train_labels_train_10_percent)

target
0    425
1    336
Name: count, dtype: int64

In [32]:
# Check the number of targets in our subset of data
train_10_percent['target'].value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [33]:
train_df['target'].value_counts()

In [34]:
# Create a Keras Layer using the USE pretrained layer from tensorflow hub
import tensorflow_hub as hub
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape= [], # empty because it is variable input length (any length)
                                        dtype = tf.string,
                                        trainable= False,
                                        name = "USE")

Model: "model_7_USE_10_percent"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense (Dense)               (None, 64)                32832     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256830721 (979.73 MB)
Trainable params: 32897 (128.50 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________


In [35]:
# Create model using the Sequential API
model_7 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64, activation = "relu"),
    layers.Dense(1, activation = "sigmoid")
], name = "model_7_USE_10_percent")

# Compile
model_7.compile(loss = 'binary_crossentropy',
                optimizer= tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

model_7.summary()


Epoch 1/5
24/24 [==============================] - 2s 22ms/step - loss: 0.6757 - accuracy: 0.6636 - val_loss: 0.6391 - val_accuracy: 0.8136
Epoch 2/5
24/24 [==============================] - 0s 8ms/step - loss: 0.6064 - accuracy: 0.8108 - val_loss: 0.5580 - val_accuracy: 0.8215
Epoch 3/5
24/24 [==============================] - 0s 8ms/step - loss: 0.5251 - accuracy: 0.8160 - val_loss: 0.4848 - val_accuracy: 0.8228
Epoch 4/5
24/24 [==============================] - 0s 8ms/step - loss: 0.4643 - accuracy: 0.8081 - val_loss: 0.4311 - val_accuracy: 0.8202
Epoch 5/5
24/24 [==============================] - 0s 8ms/step - loss: 0.4195 - accuracy: 0.8226 - val_loss: 0.3976 - val_accuracy: 0.8373


In [36]:
model_7_history = model_7.fit(train_sentences_train_10_percent,
                              train_labels_train_10_percent,
                              epochs = 5,
                              validation_data=(val_sentences, val_labels))

24/24 [==============================] - 0s 4ms/step


In [37]:
# Make predictions with the model trained on 10% of the data
model_7_pred_probs = model_7.predict(val_sentences)
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))

In [38]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

model_7_accuracy_score = accuracy_score(val_labels, model_7_preds)
model_7_precision, model_7_recall, model_7_f1score, _, =precision_recall_fscore_support(val_labels, model_7_preds, average = "weighted")

{'model_7_accuracy_score': 0.8372703412073491,
 'model_7_precision': 0.8370524103039156,
 'model_7_recall': {0.8372703412073491},
 'model_7_f1score': {0.8367856456967269}}

In [ ]:
dict_result = {"model_7_accuracy_score": model_7_accuracy_score,
               "model_7_precision": model_7_precision,
               "model_7_recall": {model_7_recall},
               "model_7_f1score": {model_7_f1score}}
dict_result